# [Модуль "Pythob для аналитиков данных"](https://ru.hexlet.io/programs/python-for-data-analysts)

## [Проект: Дашборд конверсий](https://ru.hexlet.io/programs/python-for-data-analysts/projects/100)


In [11]:
from dotenv import load_dotenv
import pandas as pd
import os
import re
import requests

#### Данные (образец):

[Визиты](https://drive.google.com/file/d/1QosQQ4RRNR9rkL4t7sB707h2Uy0XfYJe/view?usp=drive_link) - тысяча записей с визитами

[Регистрации](https://drive.google.com/file/d/1AeQz0kaSgz0lxYSDtuNm36muhy5fRCzZ/view?usp=drive_link) - тысяча записей о первых регистрациях


## Скачивание данных

In [26]:
load_dotenv()

DATE_BEGIN = os.getenv('DATE_BEGIN')
DATE_END = os.getenv('DATE_END')
API_URL = os.getenv('API_URL')

In [17]:
def download_from_api(url, endpoint, path_to_save, start_date=None, end_date=None):
    download_link = f'{url}/{endpoint}'
    if start_date or end_date:
        download_link += "?"
        if start_date:
            download_link += f'begin={start_date}'
        if end_date:
            download_link += f'&end={end_date}' if start_date else f'end={end_date}'
            
    print(f'Download from the link {download_link}...')
    response = requests.get(download_link)
    if response.ok:
        with open(path_to_save, 'wb') as file:
            file.write(response.content)
        print(f"File successfully saved as '{path_to_save}'")
    else:
        print(f"Error downloading file: status code {response.status_code}")

In [19]:
visits_path = './data/visits.json'
registrations_path = './data/registrations.json'

#### Скачивание данных **visits**

In [28]:
download_from_api(url=API_URL,
         endpoint='visits',
         path_to_save=visits_path,
         start_date=DATE_BEGIN,
         end_date=DATE_END)

Download from the link https://data-charts-api.hexlet.app/visits?begin=2023-03-01&end=2023-09-01...
File successfully saved as './data/visits.json'


#### Скачивание данных **registrations**

In [31]:
download_from_api(url=API_URL,
         endpoint='registrations',
         path_to_save=registrations_path,
         start_date=DATE_BEGIN,
         end_date=DATE_END)

Download from the link https://data-charts-api.hexlet.app/registrations?begin=2023-03-01&end=2023-09-01...
File successfully saved as './data/registrations.json'


## Изучение датафреймов

### visits

In [241]:
visits = pd.read_json(visits_path)
visits.head(3)

,visit_id,platform,user_agent,datetime
0,1de9ea66-70d3-4a1f-8735-df5ef7697fb9,web,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2023-03-01 10:36:22
1,f149f542-e935-4870-9734-6b4501eaf614,web,Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7...,2023-03-01 06:25:00
2,08f0ebd4-950c-4dd9-8e97-b5bdf073eed1,web,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,2023-03-01 10:26:13


In [242]:
visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263459 entries, 0 to 263458
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   visit_id    263459 non-null  object        
 1   platform    263459 non-null  object        
 2   user_agent  263459 non-null  object        
 3   datetime    263459 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 8.0+ MB


In [243]:
visits.describe()

,datetime
count,263459
mean,2023-06-01 06:22:22.042044672
min,2023-03-01 00:00:43
25%,2023-04-13 19:45:03
50%,2023-06-03 12:23:27
75%,2023-07-17 22:09:08
max,2023-08-31 23:52:57


In [244]:
visits.head(3)

,visit_id,platform,user_agent,datetime
0,1de9ea66-70d3-4a1f-8735-df5ef7697fb9,web,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2023-03-01 10:36:22
1,f149f542-e935-4870-9734-6b4501eaf614,web,Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7...,2023-03-01 06:25:00
2,08f0ebd4-950c-4dd9-8e97-b5bdf073eed1,web,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,2023-03-01 10:26:13


### registrations

In [229]:
registrations = pd.read_json(registrations_path)
registrations.head(3)

,datetime,user_id,email,platform,registration_type
0,2023-03-01 07:40:13,2e0f6bb8-b029-4f45-a786-2b53990d37f1,ebyrd@example.org,web,google
1,2023-03-01 13:14:00,f007f97c-9d8b-48b5-af08-119bb8f6d9b6,knightgerald@example.org,web,email
2,2023-03-01 03:05:50,24ff46ae-32b3-4a74-8f27-7cf0b8f32f15,cherylthompson@example.com,web,apple


In [231]:
registrations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21836 entries, 0 to 21835
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   datetime           21836 non-null  datetime64[ns]
 1   user_id            21836 non-null  object        
 2   email              21836 non-null  object        
 3   platform           21836 non-null  object        
 4   registration_type  21836 non-null  object        
dtypes: datetime64[ns](1), object(4)
memory usage: 853.1+ KB


In [233]:
registrations.describe()

,datetime
count,21836
mean,2023-05-29 10:17:02.002747904
min,2023-03-01 00:04:47
25%,2023-04-12 01:49:47.500000
50%,2023-05-26 13:22:09
75%,2023-07-15 17:26:30.500000
max,2023-08-31 23:43:26


In [235]:
registrations.head(3)

,datetime,user_id,email,platform,registration_type
0,2023-03-01 07:40:13,2e0f6bb8-b029-4f45-a786-2b53990d37f1,ebyrd@example.org,web,google
1,2023-03-01 13:14:00,f007f97c-9d8b-48b5-af08-119bb8f6d9b6,knightgerald@example.org,web,email
2,2023-03-01 03:05:50,24ff46ae-32b3-4a74-8f27-7cf0b8f32f15,cherylthompson@example.com,web,apple


## Расчет метрик

* Сгруппируйте данные визитов по датам и платформам
* Сгруппируйте также данные регистраций по датам и платформам
* Объедините датайфреймы, сделайте итоговый датафрейм с расчетом конверсии
* Сохраните датафрейм в формате JSON conversion.json
* Поля датафрейма:
  - date_group - сагрегированный столбец дат
  - platform - платформа (android,web,ios)
  - visits - визиты за период date_group
  - registrations - регистрации за период date_group
  - conversion - конверсия по платформе

#### Группировка данных визитов по датам и платформам

#### Подготовка данных:
* преобразование datetime -> date
* удаление дубликатов записей с одинаковым id
* удаление записей визитов ботов

In [251]:
def prepare_visits_data(visits):
    # Визиты ботов не должны влиять на расчет конверсии
    prepared_visits = visits[~visits.user_agent.str.contains('bot')].copy()
    
    # Преобразование datetime -> date
    prepared_visits['date'] = pd.to_datetime(prepared_visits.pop('datetime')).dt.date

    # Пользователи могли заходить на сайт несколько раз, прежде чем зарегистрироваться.
    # Оставить последний визит для каждого visit_id
    prepared_visits.sort_values(by=['visit_id', 'date'], ascending=[True, False], inplace=True)
    prepared_visits = prepared_visits.drop_duplicates(subset='visit_id', keep='first')
    
    prepared_visits.sort_values(by='date', inplace=True)
    return prepared_visits

In [271]:
def prepare_registrations_data(registrations):
    # Преобразование datetime -> date
    prepared_registrations = registrations.copy()
    prepared_registrations['date'] = pd.to_datetime(registrations.datetime).dt.date
    prepared_registrations.sort_values(by='date', inplace=True)
    return prepared_registrations

In [253]:
prepared_visits = prepare_visits_data(visits)
prepared_visits

,visit_id,platform,user_agent,date
89,58b3098c-99b9-4557-a7bc-ee9f5e993811,web,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,2023-03-01
159,d698a72c-5ce4-438d-8afd-1877527e1ca8,web,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2023-03-01
58,3480251a-dbd2-4b62-ae57-a9865c7b2cec,web,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2023-03-01
93,77c7c9e9-b9bd-4b68-8bad-a6a0cd24f4ec,web,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2023-03-01
299,ac66d9be-d4a0-4e1f-8093-c09c444fc90a,android,Mozilla/5.0 (Linux; Android 10; K) AppleWebKit...,2023-03-01
...,...,...,...,...
262104,e79d4703-6268-46c1-af17-b38950a7c5bd,web,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:1...,2023-08-31
261679,5de8fda9-4b83-4504-981c-92f2f43d70ce,web,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,2023-08-31
261619,7e4cca22-9907-43ae-a770-122f3bae8b89,web,Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) Apple...,2023-08-31
262078,245d9b78-27a7-42a9-a669-ed8c470746a0,web,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2023-08-31


In [273]:
prepared_registrations = prepare_registrations_data(registrations)
prepared_registrations

,datetime,user_id,email,platform,registration_type,date
0,2023-03-01 07:40:13,2e0f6bb8-b029-4f45-a786-2b53990d37f1,ebyrd@example.org,web,google,2023-03-01
63,2023-03-01 04:19:11,d241b05f-38b4-4eb7-9c1a-9e1caa00d83e,ingramgina@example.net,android,google,2023-03-01
62,2023-03-01 10:07:32,b612aff6-96dc-4a1b-b82a-000accb14d02,katelyn71@example.org,android,google,2023-03-01
61,2023-03-01 03:12:21,9d9524be-4358-410b-9b94-bc20a68a5352,lharris@example.net,android,email,2023-03-01
60,2023-03-01 09:06:36,2358ab48-15ea-4c59-9f2f-15a643ecf553,kevinvargas@example.com,android,email,2023-03-01
...,...,...,...,...,...,...
21744,2023-08-31 08:47:35,5f7179aa-67ca-4b3c-a813-edd3d113f787,jody50@example.org,web,google,2023-08-31
21743,2023-08-31 09:01:43,9fe0032b-1db1-47fa-b0b7-0db7fb19e579,shawnrichardson@example.net,web,email,2023-08-31
21742,2023-08-31 08:03:55,d24482b9-f2d6-430a-ac4a-fe09ece1bb34,michelesmall@example.org,web,yandex,2023-08-31
21756,2023-08-31 22:40:20,d43f5e69-9be0-498f-a5d2-ec14d86bcd1b,mannheather@example.net,web,apple,2023-08-31


#### Группировка данных визитов по датам и платформам

In [303]:
grouped_visits = prepared_visits.groupby(['date','platform']).agg(visits=('visit_id', 'size')).reset_index()
grouped_visits.rename(columns={'date': 'date_group'}, inplace=True)
grouped_visits

,date_group,platform,visits
0,2023-03-01,android,75
1,2023-03-01,ios,22
2,2023-03-01,web,279
3,2023-03-02,android,67
4,2023-03-02,ios,31
...,...,...,...
547,2023-08-30,ios,66
548,2023-08-30,web,1227
549,2023-08-31,android,57
550,2023-08-31,ios,50


#### Группировка данных регистраций по датам и платформам

In [305]:
grouped_registrations = prepared_registrations.groupby(['date','platform']).agg(registrations=('user_id', 'size')).reset_index()
grouped_registrations.rename(columns={'date': 'date_group'}, inplace=True)
grouped_registrations

,date_group,platform,registrations
0,2023-03-01,android,61
1,2023-03-01,ios,18
2,2023-03-01,web,8
3,2023-03-02,android,59
4,2023-03-02,ios,24
...,...,...,...
547,2023-08-30,ios,40
548,2023-08-30,web,34
549,2023-08-31,android,42
550,2023-08-31,ios,36


#### Объединение датафреймов и рассчет конверсии

In [312]:
conversion_df = pd.merge(grouped_visits, grouped_registrations, on=['date_group', 'platform'])
conversion_df['conversion'] = conversion_df['registrations'] / conversion_df['visits']
conversion_df.head()

,date_group,platform,visits,registrations,conversion
0,2023-03-01,android,75,61,0.813333
1,2023-03-01,ios,22,18,0.818182
2,2023-03-01,web,279,8,0.028674
3,2023-03-02,android,67,59,0.880597
4,2023-03-02,ios,31,24,0.774194


In [314]:
conversion_df.to_json('./conversion.json')